# <span style="color: crimson">Рекомендация тарифов</span>
---
**<span style="color: crimson">Заказчик</span>**: оператор мобильной связи «Мегалайн».  
**<span style="color: crimson">Цель анализа</span>**: нужно построить модель для задачи классификации, которая выберет подходящий тариф.  
**<span style="color: crimson">Датасет</span>**: данные о поведении клиентов, которые уже перешли на один из тарифов. Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.


---

<b>Описание данных:</b>  
* <span style="color: red"><b>сalls </b></span> — количество звонков
* <span style="color: red"><b>minutes </b></span> — суммарная длительность звонков в минутах,
* <span style="color: red"><b>messages </b></span> — количество sms-сообщений,
* <span style="color: red"><b>mb_used </b></span> — израсходованный интернет-трафик в Мб,
* <span style="color: red"><b>is_ultra </b></span> — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Импорт библиотек:

In [1]:
!pip3 install sklearn
!pip3 install optuna

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score 
from sklearn.dummy import DummyClassifier
import optuna
from sklearn.metrics import make_scorer, f1_score

## <span style="color: crimson">Этап 1</span> Изучение данных

In [3]:
df = pd.read_csv('./users_behavior.csv')

In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [6]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


### <center><span style="color: crimson"><b>Вывод</b></span></center>
В данных:  
1. Нет пропусков.
2. Почти нет выбросов (среднее близко к медиане).
3. Количество пользователей с тарифом Смарт в два раза больше чем пользователей тарифа Ультра.
4. В столбцах calls и messages тип float, можно его не исправлять.

## <span style="color: crimson">Этап 2</span> Разбиение данных на выборки

In [7]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)

Разбиение произошло корректно.

----

In [8]:
features_train = df_train.drop(['is_ultra'], axis=1) # создаем признаки для обучающей выборки
target_train = df_train['is_ultra'] # создаем целевой признак для обучающей выборки

In [9]:
print(features_train.shape)
print(target_train.shape)

(2571, 4)
(2571,)


В обучающей выборке целевой и остальные признаки разделены корректно.

In [10]:
features_test = df_test.drop(['is_ultra'], axis=1) # создаем признаки для тестовой выборки
target_test = df_test['is_ultra'] # создаем целевой признак для тестовой выборки

In [11]:
print(features_test.shape)
print(target_test.shape)

(643, 4)
(643,)


В тестовой выборке целевой и остальные признаки разделены корректно.

## <span style="color: crimson">Этап 3</span> Настройка моделей

### **Используемые модели:**
* Градиентный бустинг классификации
* Случайный лес
* Логистическая регрессия

**Настройка optuma**

In [12]:
# RANDOM_SEED = 666

# kfolds = KFold(n_splits=10, shuffle=True, random_state=RANDOM_SEED)

# def tune(objective):
#     study = optuna.create_study(direction="maximize")
#     study.optimize(objective, n_trials=100)

#     params = study.best_params
#     best_score = study.best_value
#     print(f"Best score: {best_score}\n")
#     print(f"Optimized parameters: {params}\n")
#     return params

---

<h3>Градиентный бустинг классификации</h3>

|Поиск гиперпараметров (optuna).

In [13]:
# %%time

# def objective(trial):
#     param = {
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 1.0),
#         "max_depth": trial.suggest_int("max_depth", 1, 15),
#         "min_samples_leaf": trial.suggest_int("min_samples_leaf", 3, 20),
#         "max_features": trial.suggest_float("max_features", 0.1, 1.0),
#         "n_estimators": trial.suggest_int("n_estimators", 1, 500),
#     }

#     model = GradientBoostingClassifier(**param)
#     scores = cross_val_score(
#         model,
#         features_train, 
#         target_train,
#         scoring='f1',
#         cv=kfolds,
#     ).mean()

#     return scores


# GBC_params = tune(objective)
# GBC = GradientBoostingClassifier(**GBC_params, random_state=RANDOM_SEED)

In [14]:
%%time

model_GBC = GradientBoostingClassifier(
    learning_rate=0.07851376072999307,
    max_depth=2,
    min_samples_leaf=16,
    max_features=0.618365770345268,
    n_estimators=234,
)
score_GBC = cross_val_score(
    model_GBC,
    features_train,
    target_train,
    scoring=make_scorer(f1_score, average="binary"),
    cv=3,
).mean()


print("f1 - Градиентный бустинг классификации: ", score_GBC)

f1 - Градиентный бустинг классификации:  0.6465256207388307
CPU times: total: 578 ms
Wall time: 570 ms


---

<h3>Случайный лес</h3>

|Поиск гиперпараметров (optuna).

In [15]:
# %%time

# def objective(trial):
#     param = {
#         "max_depth": trial.suggest_int("max_depth", 1, 100),
#         'n_estimators': trial.suggest_int("n_estimators", 1, 100),
#         'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 10),
#         'max_leaf_nodes': trial.suggest_int("max_leaf_nodes", 10, 100)
# }

#     model_RF = RandomForestClassifier(**param)
#     scores = cross_val_score(
#         model_RF,
#         features_train, 
#         target_train,
#         scoring='f1',
#         cv=kfolds,
#     ).mean()

#     return scores


# RF_params = tune(objective)
# RF = RandomForestClassifier(**RF_params, random_state=RANDOM_SEED)

In [16]:
%%time

model_RFC = RandomForestClassifier(
    max_depth=14,
    n_estimators=53,
    min_samples_leaf=5, 
    max_leaf_nodes=62
)
score_RFC = cross_val_score(
    model_RFC,
    features_train,
    target_train,
    scoring=make_scorer(f1_score, average="binary"),
    cv=3,
).mean()


print("f1 - Градиентный бустинг классификации: ", score_RFC)

f1 - Градиентный бустинг классификации:  0.6381103685318347
CPU times: total: 359 ms
Wall time: 366 ms


----

Наилучшее качество показывает **случайный лес** 0.81 , но при этом модель обучается дольше всех (2.7 сек).   
Нам важнее качество модели, так что мы выбирает случайный лес.

## <span style="color: crimson">Этап 4</span> Тестирование моделей

<h3>Случайный лес</h3>

In [17]:
model_RFC = RandomForestClassifier(
    max_depth=14,
    n_estimators=53,
    min_samples_leaf=5, 
    max_leaf_nodes=62
)
model_RFC.fit(features_train, target_train) 

A_RFC = model_RFC.score(features_test, target_test) 
F_RFC = f1_score(target_test, model_RFC.predict(features_test))  

print("Accuracy модели {}".format(A_RFC))
print("F1_score модели {}".format(F_RFC))
print()

Accuracy модели 0.7931570762052877
F1_score модели 0.6053412462908012



<h3>Градиентный бустинг классификации</h3>

In [18]:
%%time

model_GBC = GradientBoostingClassifier(
    learning_rate=0.07851376072999307,
    max_depth=2,
    min_samples_leaf=16,
    max_features=0.618365770345268,
    n_estimators=234,
)
model_GBC.fit(features_train, target_train) 

A_GBC = model_GBC.score(features_test, target_test) 
F_GBC = f1_score(target_test, model_GBC.predict(features_test))  

print("Accuracy модели {}".format(A_GBC))
print("F1_score модели {}".format(F_GBC))
print()

Accuracy модели 0.7962674961119751
F1_score модели 0.6112759643916914

CPU times: total: 250 ms
Wall time: 249 ms


<h3>Логистическая регрессия</h3>

In [19]:
%%time

model_LR = LogisticRegression(random_state=12345, solver='lbfgs') 
model_LR.fit(features_train, target_train) 

A_LR = model_LR.score(features_test, target_test) 
F_LR = f1_score(target_test, model_LR.predict(features_test)) 

print("Accuracy модели {}".format(A_LR))
print("F1_score модели {}".format(F_LR))
print()

Accuracy модели 0.7573872472783826
F1_score модели 0.37096774193548393

CPU times: total: 62.5 ms
Wall time: 62.1 ms


<h3>Случайная модель</h3>

In [20]:
model_DC = DummyClassifier(strategy="most_frequent")
model_DC.fit(features_train, target_train)

A_DC = model_DC.score(features_test, target_test) 
F_DC = f1_score(target_test, model_DC.predict(features_test))  

print("Accuracy модели {}".format(A_DC))
print("F1_score модели {}".format(F_DC))
print()

Accuracy модели 0.6951788491446346
F1_score модели 0.0



Если взять случайную величину, то мы получим вероятность в .7 (в среднем), так что наш алгоритм с вероятностью 0.78 выше по качеству.

## <span style="color: crimson">Этап 4</span> Результаты исследований

In [22]:
pd.DataFrame(
    [
        [
            "Случайный лес",
            F_RFC,
            A_RFC
        ],
        [
            "Градиентный бустинг (GradientBoostingClassifier)",
            F_GBC,
            A_GBC
        ],
        [
            "Логистическая регрессия",
            F_LR,
            A_LR
        ],
        [
            "Случайная модель",
            F_DC,
            A_DC
        ],
    ],
    columns=("Модель", "F1", "Accuracy"),
)

,Модель,F1,Accuracy
0,Случайный лес,0.605341,0.793157
1,Градиентный бустинг (GradientBoostingClassifier),0.611276,0.796267
2,Логистическая регрессия,0.370968,0.757387
3,Случайная модель,0.000000,0.695179


## <span style="color: crimson">Этап 5</span>  Общий вывод

Наилучшее качество показал **GradientBoostingClassifier** с параметрами:
* learning_rate=0.0785
* max_depth=2
* min_samples_leaf=16
* max_features=0.618
* n_estimators=234